In [32]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_csv('data_preprocessed_new.csv').to_numpy()

x =data[:,1:-2]
y=data[:,-1]

# Without Sampling

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
sampling = False

# With Sampling

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
from imblearn.under_sampling import ClusterCentroids 
from imblearn.over_sampling import RandomOverSampler

cc = ClusterCentroids(random_state=383)
ros = RandomOverSampler(random_state=383)
x_train, y_train = ros.fit_resample(x_train, y_train)
sampling = True

In [53]:
iln = len(x[0])
hln = 30
oln = 2
eta = 0.01
num_epoch = 10000

In [54]:
import torch
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(iln, hln)
        self.fc2 = nn.Linear(hln, oln)
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [55]:
model = Net()

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = eta)

In [57]:
x = torch.Tensor(x_train).float()
y = torch.Tensor(y_train).long()

In [58]:
for epoch in range(num_epoch):
    if (epoch+1)%1000==0:
        print(epoch+1)
    optimizer.zero_grad()
    out = model(x)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [25]:
x = torch.Tensor(x_test).float()
y = torch.Tensor(y_test).long()

In [26]:
out = model(x)

In [27]:
(_, predicted) = torch.max(out.data, 1)

In [28]:
predicted

tensor([0, 0, 0,  ..., 0, 0, 0])

In [29]:
print('Accuracy is:', (100 * torch.sum(y == predicted).double() / len(y)))

Accuracy is: tensor(91.6712, dtype=torch.float64)


In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
print("Accuracy:",accuracy_score(y_test, predicted))
print("F1 scores for 2 classes:",f1_score(y_test, predicted, average=None))
cm = confusion_matrix(y_test, predicted)

plt.figure(figsize = (10,7))
ax = plt.axes()
sn.heatmap(cm, annot=True,fmt='d',ax=ax)
ax.set_title('Confusion Matrix of LR {} oversampling'.format("with" if sampling == True else "without"))